In [1]:
import io
import os
# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/rittikaadhikari/Downloads/off_credentials.json"

def fcn(file_name):
    # Instantiates a client
    client = speech.SpeechClient()

    # Loads the audio into memory
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=41000,
        language_code='en-US')

    # Detects speech in the audio file
    response = client.recognize(config, audio)
    string = ""
    for result in response.results:
        string += ('Transcript: {}'.format(result.alternatives[0].transcript))
        
    return string



In [3]:
import numpy as np
import cv2
import sounddevice as sd
import soundfile as sf



cap = cv2.VideoCapture(0)
recordings = list()
num_sec = 0

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    duration = 1  # seconds
    myrecording = sd.rec(frames=int(duration * 41000),samplerate=41000, channels=1, blocking=True)
    recordings.extend(myrecording)
    num_sec += duration
    
    if (num_sec == 4): 
        sf.write("hi.wav", recordings, 41000)
        string = fcn("hi.wav")
        cv2.putText(frame, string, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
        recordings.clear()
        num_sec = 0
    
    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    # Our operations on the frame come here
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()